In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import random
import keras
from keras.models import Sequential,Model
from keras.layers import Dense, Activation, Flatten, Convolution1D, Dropout,Input,BatchNormalization,MaxPooling1D,concatenate
from keras.utils import np_utils,multi_gpu_model
from keras.callbacks import ModelCheckpoint
from keras import regularizers
from sklearn.cluster import KMeans
from scipy import stats
from sklearn import preprocessing
from keras import backend as bek
import gc
%matplotlib inline

Using TensorFlow backend.


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
"""
data cell
"""
df1=pd.read_csv('/home/dp/Desktop/end/pair_end_merge_BG_random_HR-test-v5-test.csv',index_col=0)


cond2 = df1['count']!=0
df1 = df1[cond2]

#normalize
target = 'BG'


In [4]:
def c(ref_values, pred_values):

    #Checking to see if the lengths of the reference and prediction arrays are the same
    assert (len(ref_values) == len(pred_values)), "Unequal number of values (reference : {}) (prediction : {}).".format(len(ref_values), len(pred_values))

    #Checks to see if the values are within the normal physiological range, otherwise it gives a warning
    if max(ref_values) > 400 or max(pred_values) > 400:
        print("Input Warning: the maximum reference value {} or the maximum prediction value {} exceeds the normal physiological range of glucose (<400 mg/dl).".format(max(ref_values), max(pred_values)))
    if min(ref_values) < 0 or min(pred_values) < 0:
        print("Input Warning: the minimum reference value {} or the minimum prediction value {} is less than 0 mg/dl.".format(min(ref_values),  min(pred_values)))

    #Statistics from the data
    zone = [0] * 5
    for i in range(len(ref_values)):
        if (ref_values[i] <= 70 and pred_values[i] <= 70) or (pred_values[i] <= 1.2*ref_values[i] and pred_values[i] >= 0.8*ref_values[i]):
            zone[0] += 1    #Zone A

        elif (ref_values[i] >= 180 and pred_values[i] <= 70) or (ref_values[i] <= 70 and pred_values[i] >= 180):
            zone[4] += 1    #Zone E

        elif ((ref_values[i] >= 70 and ref_values[i] <= 290) and pred_values[i] >= ref_values[i] + 110) or ((ref_values[i] >= 130 and ref_values[i] <= 180) and (pred_values[i] <= (7/5)*ref_values[i] - 182)):
            zone[2] += 1    #Zone C
        elif (ref_values[i] >= 240 and (pred_values[i] >= 70 and pred_values[i] <= 180)) or (ref_values[i] <= 175/3 and pred_values[i] <= 180 and pred_values[i] >= 70) or ((ref_values[i] >= 175/3 and ref_values[i] <= 70) and pred_values[i] >= (6/5)*ref_values[i]):
            zone[3] += 1    #Zone D
        else:
            zone[1] += 1    #Zone B
            
    confidence_score = round((zone[0]*5+zone[1]*2.5)/(len(ref_values)*5)*100,2)
    
    return confidence_score,zone

In [5]:
def c1(ref_values, pred_values):
    
    sum_a = 0
    for i in range(len(ref_values)):
        d = abs(pred_values[i]-ref_values[i])
        if((d/ref_values[i])<1):
            a = abs(d/ref_values[i]-1)
        else:
            a = 0
        sum_a = sum_a + a
    acc = sum_a/len(ref_values)
    acc = round(acc*100,2)
    return acc

In [6]:
'''
model cell
'''
def base_model():
    in_s = Input((406, 1))
    model_s = Convolution1D(nb_filter=256, filter_length=3) (in_s)
    model_s = BatchNormalization(axis=-1)(model_s)
    model_s = Activation('relu')(model_s)
    model_s = MaxPooling1D(pool_size=2)(model_s)
    
    model_s = Convolution1D(nb_filter=256, filter_length=3) (model_s)
    model_s = BatchNormalization(axis=-1)(model_s)
    model_s = Activation('relu')(model_s)
    model_s = MaxPooling1D(pool_size=2)(model_s)
    
    model_s = Convolution1D(nb_filter=512, filter_length=3) (model_s)
    model_s = BatchNormalization(axis=-1)(model_s)
    model_s = Activation('relu')(model_s)
    model_s = MaxPooling1D(pool_size=2)(model_s)
    
    model_s = Convolution1D(nb_filter=1024, filter_length=3) (model_s)
    model_s = BatchNormalization(axis=-1)(model_s)
    model_s = Activation('relu')(model_s)
    model_s = MaxPooling1D(pool_size=2)(model_s)
    
    model_s = Convolution1D(nb_filter=2048, filter_length=3) (model_s)
    model_s = BatchNormalization(axis=-1)(model_s)
    model_s = Activation('relu')(model_s)
    model_s = MaxPooling1D(pool_size=2)(model_s)
    
    model_s = Flatten()(model_s)
    
    
    m = BatchNormalization(axis=-1)(model_s)
    m = Dense(4096,activation='relu')(m)
    m = Dense(2048,activation='relu')(m)
    out = Dense(1,activation='linear')(m)
    
    model = Model(inputs=[in_s], outputs=[out])
    
    adam=keras.optimizers.Adam(lr=0.00001, beta_1=0.9, beta_2=0.999)
    model = multi_gpu_model(model,gpus=6)
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [7]:
def test_step(train_temp,test_temp):
    X_train_s = train_temp.iloc[:,7:413]
    X_test_s = test_temp.iloc[:,7:413]
    
    f1=['BG']
    Y_train = train_temp[f1]
    Y_train = scalert.transform(Y_train)
    

    X_train_r_s = np.zeros((len(X_train_s), 406, 1))
    X_train_r_s[:, :, 0] = X_train_s


    X_test_r_s = np.zeros((len(X_test_s), 406, 1))
    X_test_r_s[:, :, 0] = X_test_s
    

    model= base_model()
    model.fit([X_train_r_s], Y_train, epochs=1000, batch_size=3000,verbose=0)

    
    test_pred = model.predict([X_test_r_s])
    test_pred1 = scalert.inverse_transform(test_pred)
    test_temp['BG_p']=test_pred1
    temp_a=0
    temp_b=0
    test_BG_r=[]
    test_BG_p=[]
                        
    for i,item in enumerate(test_temp['Person No']):
        if((test_temp['Person No'].iloc[i]!=temp_a) | (test_temp['time'].iloc[i]!=temp_b)):
            
            #administrator mode
            test_BG_r.append(test_temp['BG'].iloc[i])
            temp = test_temp[(test_temp['Person No']==test_temp['Person No'].iloc[i])&(test_temp['time']==test_temp['time'].iloc[i])]
            t = np.median(temp['BG_p'])
            test_BG_p.append(t)
            temp_a = test_temp['Person No'].iloc[i]
            temp_b = test_temp['time'].iloc[i]
            
    print(test_BG_p)
    print(test_BG_r)
    del model
    bek.clear_session()
    gc.collect()
    return (test_BG_p,test_BG_r)

In [8]:
################
testingNo_all = [1]
################

for testingNo in testingNo_all:
    test_p=[]
    test_r=[]
    test_zone=[]
    acc=[]
    for test_time in range(4,16):
        test  = df1[(df1['count']==testingNo)&(df1['count_time']==test_time)]
        if(test.empty):
            continue
        else:
            test_index = test.index
            data_base = df1.drop(test_index)
            t = data_base[target]
            t = t.values.reshape(-1, 1)
            scalert = preprocessing.MinMaxScaler().fit(t)
            test_train = data_base[(data_base['count']==testingNo)&(data_base['count_time']<test_time)]
    
        
            p_1=[]
            p_2=[]
            for i in range(0,3):
                (temp_pred,temp_real) = test_step(test_train,test)
                p_1.append(temp_pred[0])
                p_2.append(temp_pred[1])
        
        
            m_pred=[np.median(p_1),np.median(p_2)]
        
            test_p.append(m_pred)
            test_r.append(temp_real)
        
            reshape_p=[]
            reshape_r=[]

            for i in range(len(m_pred)):
                reshape_p.append(m_pred[i])
            for i in range(len(temp_real)):
                reshape_r.append(temp_real[i])
                
            no_mind,test_z = c(reshape_r,reshape_p)
            test_acc = c1(reshape_r,reshape_p)
        
            acc.append(test_acc)
            test_zone.append(test_z)
        
            print("test No:",testingNo,"test time:",test_time)
            print("BG real:",temp_real," BG pred:",m_pred,"accuracy:",test_acc,"%",'test_zone:',test_z)

            
    d = {'BG_real': test_r, 'BG_pred': test_p, 'test_zone':test_zone,'accuracy':acc}
    df_f = pd.DataFrame(data = d)
    
    ind =[]
    for j in range(4,len(df_f.index)+4):
        ind.append(j)    
    df_f.index=ind
    
    name_temp =str(testingNo)+'-1000-256.csv'
    
    df_f.to_csv(name_temp)

[130.57135, 132.78973]
[143.0, 145.0]
[142.05383, 143.51746]
[143.0, 145.0]
[139.96185, 141.14078]
[143.0, 145.0]
test No: 1888 test time: 4
BG real: [143.0, 145.0]  BG pred: [139.96185, 141.14078] accuracy: 97.61 % test_zone: [2, 0, 0, 0, 0]
[126.233826, 126.17375]
[123.0, 118.0]
[158.9032, 178.33334]
[123.0, 118.0]
[148.16254, 149.78676]
[123.0, 118.0]
test No: 1888 test time: 5
BG real: [123.0, 118.0]  BG pred: [148.16254, 149.78676] accuracy: 76.3 % test_zone: [0, 2, 0, 0, 0]
[135.77747, 133.16124]
[143.0, 136.0]
[135.33769, 134.09383]
[143.0, 136.0]
[131.61754, 128.78467]
[143.0, 136.0]
test No: 1888 test time: 6
BG real: [143.0, 136.0]  BG pred: [135.33769, 133.16124] accuracy: 96.28 % test_zone: [2, 0, 0, 0, 0]
[124.25681, 122.385765]
[117.0, 118.0]
[116.96293, 116.95208]
[117.0, 118.0]
[126.387596, 124.61996]
[117.0, 118.0]
test No: 1888 test time: 7
BG real: [117.0, 118.0]  BG pred: [124.25681, 122.385765] accuracy: 95.04 % test_zone: [2, 0, 0, 0, 0]
[126.589874, 125.87587]
[1

[154.9436, 159.3931]
[150.0, 147.0]
[173.61763, 153.16216]
[150.0, 147.0]
[202.09343, 198.41086]
[150.0, 147.0]
test No: 2203 test time: 6
BG real: [150.0, 147.0]  BG pred: [173.61763, 159.3931] accuracy: 87.91 % test_zone: [2, 0, 0, 0, 0]
[163.17233, 159.63553]
[192.0, 183.0]
[191.45064, 182.10812]
[192.0, 183.0]
[167.29393, 159.50613]
[192.0, 183.0]
test No: 2203 test time: 7
BG real: [192.0, 183.0]  BG pred: [167.29393, 159.63553] accuracy: 87.18 % test_zone: [2, 0, 0, 0, 0]
[192.29094, 214.67703]
[161.0, 151.0]
[172.49982, 191.77304]
[161.0, 151.0]
[181.92986, 195.92822]
[161.0, 151.0]
test No: 2203 test time: 8
BG real: [161.0, 151.0]  BG pred: [181.92986, 195.92822] accuracy: 78.62 % test_zone: [1, 1, 0, 0, 0]
[227.56535, 226.72484]
[190.0, 184.0]
[220.67163, 218.3786]
[190.0, 184.0]
[232.49307, 236.31]
[190.0, 184.0]
test No: 2204 test time: 4
BG real: [190.0, 184.0]  BG pred: [227.56535, 226.72484] accuracy: 78.5 % test_zone: [1, 1, 0, 0, 0]
[196.29332, 204.02026]
[176.0, 164.0

[152.7268, 147.31166]
[138.0, 147.0]
[181.05255, 176.63391]
[138.0, 147.0]
test No: 689 test time: 6
BG real: [138.0, 147.0]  BG pred: [152.7268, 149.9572] accuracy: 93.66 % test_zone: [2, 0, 0, 0, 0]
[164.19296, 143.9381]
[141.0, 152.0]
[144.1056, 136.85692]
[141.0, 152.0]
[154.3763, 158.43332]
[141.0, 152.0]
test No: 689 test time: 7
BG real: [141.0, 152.0]  BG pred: [154.3763, 143.9381] accuracy: 92.6 % test_zone: [2, 0, 0, 0, 0]
[90.93266, 33.019257]
[122.0, 128.0]
[105.08878, 92.24191]
[122.0, 128.0]
[72.8665, 185.03601]
[122.0, 128.0]
test No: 756 test time: 4
BG real: [122.0, 128.0]  BG pred: [90.93266, 92.24191] accuracy: 73.3 % test_zone: [0, 2, 0, 0, 0]
[102.336174, 81.47705]
[93.0, 92.0]
[111.502914, 90.045525]
[93.0, 92.0]
[99.83307, 82.519196]
[93.0, 92.0]
test No: 756 test time: 5
BG real: [93.0, 92.0]  BG pred: [102.336174, 82.519196] accuracy: 89.83 % test_zone: [2, 0, 0, 0, 0]
[100.03516, 93.462204]
[179.0, 193.0]
[98.76547, 93.439545]
[179.0, 193.0]
[98.82893, 93.9571